# Методы отбора признаков в задачах Uplift-моделирования, основанные на важности признаков

В данном ноутбуке содержится исследование методов отбора признаков, основанных на важности признаков:
1) Отбор на основе shap-значений (shap > 0)
2) Отбор на основе permutation-importance (PI > 0)
3) Отбор на основе комбинации shap > 0 и PI > 0
4) Отбор с помощью feature importance из UpliftTree.

Во всех случаях рассчитываются значения важности признаков, после чего они подаются в T-Learner, базовой моделью которого является CatBoostClassifier

In [1]:
!pip install scikit-uplift

In [2]:
!pip install causalml

In [3]:
!pip install catboost

In [4]:
from sklift.datasets import fetch_lenta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

from causalml.metrics import plot_gain, auuc_score
from causalml.inference.tree import UpliftTreeClassifier

from catboost import CatBoostClassifier, Pool
from sklearn.inspection import permutation_importance
import shap
import numpy as np
import pandas as pd
from causalml.metrics import auuc_score

ERROR:duecredit:Failed to import duecredit due to No module named 'duecredit'


In [5]:
SEED = 42

In [6]:
dataset = fetch_lenta()
data, target, treatment = dataset.data, dataset.target, dataset.treatment

### Загрузка и предобработка данных

После проведения EDA пришли к следующим выводам:

1) В датасете присутсвует дисбаланс классов: число записей, поповших в тестовую выбоку (воздействие было) составляет 75%, а контрольная группа составляет 25%.

2) Так же в датасете есть дисбаланс таргета: всего лишь 10% покупателей совершили целевое дейсвтие, в то время как другие 90% этого дейсвтия не совершали.

3) В датасете есть пропуски - их нужно заполнить. Так как минимальное значение в датасете - 0, заполним пропсуски значением -100.

4) В датасете есть только один категориальный признак - пол клиента, в данном признаке есть пропуски. Для данного признака присвоим значение 1, если пол мужской, 0, если женский. Пропущенные значения заполним -1.

5) В датасете есть 30 полных дубликатов, от них необходимо избавиться.

6) В результате превичного анализа датаффрейма никакие признаки не были удалены.

Предобработаем данные с учетом этих выводов:

In [7]:
# Преобразование признака gender
data['gender'] = data['gender'].map({'Male': 1, 'Female': 0})
data['gender'] = data['gender'].fillna(-1)

In [8]:
# Заполнение пропусков
data = data.fillna(-100)

In [9]:
data.isna().sum().sum()

np.int64(0)

In [10]:
# Удаление явных дубликатов
data = data.drop_duplicates()

In [11]:
# Разбиение на train, valid, test выборки

data['treatment'] = treatment
data['treatment'] = (data['treatment'] == 'test').astype(int)
data['target'] = target

train_val_idx, test_idx = train_test_split(data.index, test_size=0.2, random_state=SEED, stratify=data[['treatment', 'target']])

train_idx, val_idx = train_test_split(train_val_idx, test_size=0.25, random_state=SEED, stratify=data.loc[train_val_idx, ['treatment', 'target']])

X_train = data.drop(columns=['treatment', 'target']).loc[train_idx]
X_val = data.drop(columns=['treatment', 'target']).loc[val_idx]
X_test = data.drop(columns=['treatment', 'target']).loc[test_idx]

treatment_train = data['treatment'][train_idx]
treatment_val = data['treatment'][val_idx]
treatment_test = data['treatment'][test_idx]

y_train = target[train_idx]
y_val = target[val_idx]
y_test = target[test_idx]

### Отбор признаков

Для начала отберем признаки с помощью shap, permutation importance и объединении этих методов. После чего построим T-Learner на основе CatBoost

In [38]:
def select_features_shap(model, X, y):
  explainer = shap.TreeExplainer(model)
  shap_values = explainer.shap_values(X)

  shap_mean = shap_values.mean(axis=0)
  features = pd.DataFrame({'feature': X.columns, 'importance': shap_mean})
  features = features[features.importance > 0]
  features = features.sort_values('importance', ascending=False)

  return features['feature'].tolist()


In [13]:
def select_features_permutation(model, X, y, n_repeats=5):

    result = permutation_importance(model, X, y, n_repeats=n_repeats, random_state=SEED)

    perm_imp = pd.DataFrame({'feature': X.columns, 'importance': result.importances_mean})
    perm_imp = perm_imp[perm_imp.importance > 0]
    perm_imp = perm_imp.sort_values('importance', ascending=False)

    return perm_imp['feature'].tolist()

In [23]:
def select_features_combined(model, X, y, top_k=None):

    shap_feats = set(select_features_shap(model, X, y))
    perm_feats = set(select_features_permutation(model, X, y))

    combined = list(shap_feats.intersection(perm_feats))

    return combined

In [39]:
def train_t_learner(X_train, y_train, treatment_train, X_test, y_test, treatment_test, top_k=None, selection_method='combined'):

    X_c = X_train[treatment_train == 0]
    y_c = y_train[treatment_train == 0]
    X_t = X_train[treatment_train == 1]
    y_t = y_train[treatment_train == 1]


    model_control_fs = CatBoostClassifier(
        loss_function='Logloss',
        verbose=False,
        random_state=SEED
    )
    model_control_fs.fit(X_c, y_c)

    if selection_method == 'shap':
        features_c = select_features_shap(model_control_fs, X_c, y_c)
    elif selection_method == 'permutation':
        features_c = select_features_permutation(model_control_fs, X_c, y_c)
    else:
        features_c = select_features_combined(model_control_fs, X_c, y_c)

    model_treatment_fs = CatBoostClassifier(
        loss_function='Logloss',
        verbose=False,
        random_state=SEED
    )
    model_treatment_fs.fit(X_t, y_t)

    if selection_method == 'shap':
        features_t = select_features_shap(model_treatment_fs, X_t, y_t)

    elif selection_method == 'permutation':
        features_t = select_features_permutation(model_treatment_fs, X_t, y_t)

    else:
        features_t = select_features_combined(model_treatment_fs, X_t, y_t)

    final_model_control = CatBoostClassifier(
        loss_function='Logloss',
        verbose=False,
        random_state=SEED
    )
    final_model_control.fit(X_c[features_c], y_c)

    final_model_treatment = CatBoostClassifier(
        loss_function='Logloss',
        verbose=False,
        random_state=SEED
    )
    final_model_treatment.fit(X_t[features_t], y_t)


    pred_c = final_model_control.predict_proba(X_test[features_c])[:, 1]
    pred_t = final_model_treatment.predict_proba(X_test[features_t])[:, 1]

    uplift_pred = pred_t - pred_c

    res_data = pd.DataFrame({'uplift': uplift_pred,
                       'y': y_test,
                       'treatment': treatment_test})

    auuc = auuc_score(res_data, outcome_col='y', treatment_col='treatment')

    return auuc, features_c, features_t, uplift_pred

In [40]:
auuc_shap, feats_control_shap, feats_treatment_shap, uplift_pred_shap = train_t_learner(
    X_train, y_train, treatment_train,
    X_test, y_test, treatment_test,
    selection_method='shap'
)

print('AUUC:', auuc_shap)
print('Число признаков для control модели:', len(feats_control_shap))
print('Число признаков для treatment модели:', len(feats_treatment_shap))


AUUC: uplift    0.493202
dtype: float64
Число признаков для control модели: 97
Число признаков для treatment модели: 92


In [21]:
auuc_pi, feats_control_pi, feats_treatment_pi, uplift_pred_pi = train_t_learner(
    X_train, y_train, treatment_train,
    X_test, y_test, treatment_test,
    selection_method='permutation'
)

print('AUUC:', auuc_pi)
print('Число признаков для control модели:', len(feats_control_pi))
print('Число признаков для treatment модели:', len(feats_treatment_pi))

AUUC: uplift    0.563487
dtype: float64
Число признаков для control модели: 173
Число признаков для treatment модели: 179


In [41]:
auuc_combined, feats_control_combined, feats_treatment_combined, uplift_pred_combined = train_t_learner(
    X_train, y_train, treatment_train,
    X_test, y_test, treatment_test,
    selection_method='combined'
)

print('AUUC:', auuc_combined)
print('Число признаков для control модели:', feats_control_combined)
print('Число признаков для treatment модели:', feats_treatment_combined)

AUUC: uplift    0.446839
dtype: float64
Число признаков для control модели: ['cheque_count_12m_g20', 'k_var_sku_price_6m_g42', 'sale_count_6m_g32', 'cheque_count_12m_g41', 'k_var_sku_price_15d_g34', 'k_var_disc_share_6m_g27', 'sale_count_6m_g57', 'k_var_disc_share_1m_g44', 'k_var_disc_share_3m_g27', 'cheque_count_6m_g32', 'k_var_sku_per_cheque_15d', 'cheque_count_3m_g25', 'cheque_count_12m_g21', 'k_var_sku_price_3m_g48', 'sale_sum_6m_g32', 'age', 'cheque_count_6m_g56', 'k_var_count_per_cheque_1m_g27', 'k_var_sku_price_3m_g32', 'main_format', 'k_var_disc_share_1m_g27', 'k_var_count_per_cheque_6m_g44', 'crazy_purchases_cheque_count_3m', 'k_var_days_between_visits_1m', 'k_var_sku_price_6m_g49', 'k_var_sku_price_3m_g33', 'cheque_count_6m_g48', 'k_var_disc_share_3m_g38', 'sale_count_6m_g33', 'stdev_discount_depth_15d', 'k_var_disc_share_1m_g49', 'sale_sum_6m_g54', 'sale_sum_12m_g25', 'cheque_count_6m_g46', 'crazy_purchases_cheque_count_6m', 'cheque_count_12m_g32', 'sale_count_6m_g24', 'k_va

In [42]:
len(feats_control_combined), len(feats_treatment_combined)

(89, 90)

Теперь получим важность признаков с помощью встроенного в UpliftTree метода feature_importances_:

In [28]:
treatment_train_str = treatment_train.astype(str)
treatment_test_str = treatment_test.astype(str)

uplift_tree = UpliftTreeClassifier(
    max_depth=5,
    min_samples_leaf=100,
    min_samples_treatment=50,
    n_reg=100,
    evaluationFunction='KL',
    control_name='0',
    random_state=SEED
)

uplift_tree.fit(X_train.values, treatment_train_str.values, y_train.values)

In [29]:
feature_importances = pd.Series(uplift_tree.feature_importances_, index=X_train.columns)

selected_features = feature_importances[feature_importances > 0].index.tolist()

print(f'Число отобранных признаков {len(selected_features)}')

Число отобранных признаков 14


In [30]:
selected_features

['cheque_count_12m_g33',
 'cheque_count_12m_g45',
 'cheque_count_3m_g20',
 'crazy_purchases_cheque_count_3m',
 'k_var_count_per_cheque_3m_g24',
 'k_var_count_per_cheque_6m_g27',
 'k_var_days_between_visits_15d',
 'k_var_disc_share_3m_g44',
 'k_var_sku_price_15d_g34',
 'k_var_sku_price_1m_g34',
 'k_var_sku_price_3m_g27',
 'response_sms',
 'response_viber',
 'sale_sum_3m_g24']

In [31]:
X_c = X_train[treatment_train == 0][selected_features]
y_c = y_train[treatment_train == 0]

X_t = X_train[treatment_train == 1][selected_features]
y_t = y_train[treatment_train == 1]

model_c = CatBoostClassifier(
    loss_function='Logloss',
    verbose=False,
    random_state=SEED
)
model_c.fit(X_c, y_c)

model_t = CatBoostClassifier(
    loss_function='Logloss',
    verbose=False,
    random_state=SEED
)
model_t.fit(X_t, y_t)

In [32]:
X_test_sel = X_test[selected_features]

pred_c = model_c.predict_proba(X_test_sel)[:, 1]
pred_t = model_t.predict_proba(X_test_sel)[:, 1]

uplift_pred = pred_t - pred_c

In [35]:
res_data = pd.DataFrame({'uplift': uplift_pred,
                       'y': y_test,
                       'treatment': treatment_test})

auuc = auuc_score(res_data, outcome_col='y', treatment_col='treatment')
print('AUUC:', auuc)

AUUC: uplift    0.713305
dtype: float64


**Вывод:**

1. Для данной части исследования использовали следующие методы отбора признаков:


*   Shap importance - отдельно для treatment и control моделей были рассчитаны shap значения всех признаков. После чего были оставлены только те признаки, shap-важность которых была больше 0
*   Permutation importance - аналогично предыдущему методу, только в качестве оценки важности признаков использовался метод permutation importance

* Shap + permutation - использовалось пересечение двух указанных выше методов

* UpliftTree - построили UpliftTreeClassifier, получили из него важность признаков. Оставили только те признаки, важность которых > 0.

После чего построили T-Learner на основе CatBoost на отобранных признаках.


2. Качество представленных методов отбора признаков значительно выше фильтрационных методов (где максимаьное значение auuc составляело 0.38)

3.  Ниже представлена таблица с метрикой auuc для реализованных методов и число вошедших признаков:


|Метод отбора признаков| AUUC | Число признаков treatment | Число признаков control |
|------| ----| --- | --- |
|Shap| 0.493202| 92 | 97 |
|Permutation| 0.563487| 179 | 173 |
|Shap + Permutation| 0.446839 | 90 | 89|
|UpliftTree importance| 0.713305| 14 | 14 |

Для последнего метода метрика auuc значительно выше.